In [20]:
import geopandas as gpd
import pandas as pd
import json
from pandas_ods_reader import read_ods

In [24]:
df = read_ods("data/CO2.ods", 1)
df.head()

,unnamed.1,unnamed.2,unnamed.3,unnamed.4,unnamed.5,unnamed.6,unnamed.7,unnamed.8,unnamed.9,unnamed.10,unnamed.11,unnamed.12
0,State,"Sum of 2,005","Sum of 2,006","Sum of 2,007","Sum of 2,008","Sum of 2,009","Sum of 2,010","Sum of 2,011","Sum of 2,012","Sum of 2,013","Sum of 2,014","Sum of 2,015"
1,None,2.5485e+08,2.6186e+08,2.67425e+08,2.67297e+08,2.64355e+08,2.66278e+08,2.70057e+08,2.91736e+08,2.92003e+08,2.40837e+08,2.43122e+08
2,Andaman and Nicobar Island,-7.10765e+06,-7.10741e+06,-7.10464e+06,-7.10861e+06,-7.11223e+06,-7.11409e+06,-7.12729e+06,-1.17063e+06,-1.17083e+06,357447,360883
3,Andhra Pradesh,5.34937e+06,5.84013e+06,6.13497e+06,6.42158e+06,5.46531e+06,6.5453e+06,6.22658e+06,-8.24616e+06,-9.63313e+06,-1.89579e+07,-1.97201e+07
4,Arunachal Pradesh,-2.06442e+07,-2.06566e+07,-2.06603e+07,-2.09495e+07,-2.09422e+07,-2.09163e+07,-2.07493e+07,5.46554e+06,5.48829e+06,6.42235e+06,6.27668e+06


In [25]:
def preprocess(df):
    cols = ['State']
    for d in df.columns:
        x = df[d][0]
    #     print(x)
        if(x!='State'):
            y = x.split(" ")
            cols.append(y[3])
    df.columns = cols
    state_yr = []
    state = ''
    for i in range(2,len(df)):
        for col in cols:
            if(col =='State'):
                state = df[col][i]
            else:
                year = col
                val = df[col][i]
    #             print(state, year,val)
                state_yr.append([state, year,val])
    df = pd.DataFrame(state_yr)
    df.head()
    df.columns = ['state','year','carbon_emission']
    return df


In [26]:
def delete_null(df):
    df = df.drop(df.index[0])
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

In [27]:
df = preprocess(df)

In [36]:
df = delete_null(df)


In [38]:
df

,state,year,carbon_emission
0,Andaman and Nicobar Island,"2,006",-7.107412e+06
1,Andaman and Nicobar Island,"2,007",-7.104641e+06
2,Andaman and Nicobar Island,"2,008",-7.108615e+06
3,Andaman and Nicobar Island,"2,009",-7.112226e+06
4,Andaman and Nicobar Island,"2,010",-7.114089e+06
...,...,...,...
401,None,"2,011",2.700570e+08
402,None,"2,012",2.917356e+08
403,None,"2,013",2.920030e+08
404,None,"2,014",2.408371e+08


In [47]:
from bokeh.io import output_notebook, show
from bokeh.palettes import brewer
output_notebook()

Loading BokehJS ...

In [56]:
# Plot a complex chart with intearctive hover in a few lines of code

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
# from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.state = df.state.astype(str)
df.year = df.year.astype(str)
# df.year = df.year.astype(str)
# print(df)
group = df.groupby(by=['state', 'year'])
group.head()
source = ColumnDataSource(group)

p = figure(plot_width=11500, plot_height=1000, title="State wise Carbon emissions",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "state"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('state_year', palette=inferno(30), 
                         factors=sorted(df.year.unique()), end=1)

p.vbar(x='state_year', top='carbon_emission_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("Carbon", "@carbon_emission_mean"), ("state, year", "@state_year")]))

show(p)